# SQL Analysis: Genre Engagement Patterns

## Objective
Analyze whether different game genres exhibit unequal player engagement patterns using SQL queries.

## Research Question
**Do different game genres exhibit unequal player engagement patterns?**

## Dataset
- **Input:** `archive1/games_march2025_ml_ready.csv` (pre-cleaned ML-ready dataset)
- **Filters Applied:** 
  - Games with ≥500 total reviews (already filtered in ML-ready dataset)
  - Paid games only (price > 0) - applied in this notebook


## Setup and Imports


In [10]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


In [11]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark Session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("GenreEngagementAnalysis") \
    .getOrCreate()

# Set log level to reduce output noise
spark.sparkContext.setLogLevel("WARN")

print("Spark session created")


Spark session created


25/12/26 03:40:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/26 03:40:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/12/26 03:40:09 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/12/26 03:40:09 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


## Step 1: Load Pre-cleaned ML-Ready Data

**Note:** We use the existing ML-ready dataset which already has:
- Unnecessary columns removed
- Games with ≥500 reviews filtered
- Boolean columns converted to 0/1
- Null values handled

We only need to filter for paid games (price > 0).


In [12]:
# Read pre-cleaned ML-ready CSV file
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("escape", '"') \
    .option("multiLine", "true") \
    .option("quote", '"') \
    .option("ignoreLeadingWhiteSpace", "true") \
    .option("ignoreTrailingWhiteSpace", "true") \
    .csv("archive1/games_march2025_ml_ready.csv")

print(f"ML-Ready dataset loaded:")
print(f"Total games: {df.count():,}")
print(f"Total columns: {len(df.columns)}")
print(f"\nNote: This dataset already has:")
print(f"  - Unnecessary columns removed")
print(f"  - Games with ≥500 reviews filtered")
print(f"  - Null values handled")


ML-Ready dataset loaded:
Total games: 11,889
Total columns: 31

Note: This dataset already has:
  - Unnecessary columns removed
  - Games with ≥500 reviews filtered
  - Null values handled


### Step 1.1: Filter Paid Games Only


In [13]:
# Filter for paid games only (price > 0)
games_before = df.count()
print(f"Games before filtering: {games_before:,}")

df_cleaned = df.filter(
    (col('price').isNotNull()) &
    (col('price') > 0)  # Remove free games
)

games_after = df_cleaned.count()
games_removed = games_before - games_after

print(f"\n- Paid games (price > 0): {games_after:,}")
print(f"- Free games removed: {games_removed:,} ({games_removed/games_before*100:.2f}%)")


Games before filtering: 11,889

- Paid games (price > 0): 9,460
- Free games removed: 2,429 (20.43%)


### Step 1.2: Create SQL View


In [14]:
# Create temporary view for SQL queries
df_cleaned.createOrReplaceTempView("games")

print("Data ready and registered as 'games' view")

print(f"\n- Total games: {df_cleaned.count():,}")
print(f"- All games have ≥500 reviews (from ML-ready dataset)")
print(f"- All games are paid (price > 0)")


Data ready and registered as 'games' view

- Total games: 9,460
- All games have ≥500 reviews (from ML-ready dataset)
- All games are paid (price > 0)



# Genre Engagement Analysis

## Query: Genre Engagement Patterns by Playtime


In [ ]:
# Genre Engagement Query
query_genre_engagement = """
WITH parsed_genres AS (
    SELECT
        appid,
        TRIM(genre) AS genre,
        average_playtime_forever / 60.0 AS playtime_hours
    FROM games
    LATERAL VIEW explode(
        split(
            translate(genres, '[]\\'', ''),
            ','
        )
    ) AS genre
    WHERE
        genre IS NOT NULL
        AND TRIM(genre) != ''
        AND average_playtime_forever IS NOT NULL
        AND average_playtime_forever > 0
),
genre_engagement AS (
    SELECT
        genre,
        COUNT(*) AS game_count,
        AVG(playtime_hours) AS avg_playtime_hours,
        PERCENTILE_APPROX(playtime_hours, 0.50) AS median_playtime_hours,
        PERCENTILE_APPROX(playtime_hours, 0.75) AS p75_playtime_hours,
        PERCENTILE_APPROX(playtime_hours, 0.25) AS p25_playtime_hours,
        STDDEV(playtime_hours) AS stddev_playtime_hours
    FROM parsed_genres
    GROUP BY genre
    HAVING COUNT(*) >= 50
)
SELECT
    genre,
    game_count,
    ROUND(avg_playtime_hours, 2) AS avg_playtime_hours,
    ROUND(median_playtime_hours, 2) AS median_playtime_hours,
    ROUND(p75_playtime_hours, 2) AS p75_playtime_hours,
    ROUND(p25_playtime_hours, 2) AS p25_playtime_hours,
    ROUND(stddev_playtime_hours, 2) AS stddev_playtime_hours
FROM genre_engagement
ORDER BY median_playtime_hours DESC
"""

spark.sql(query_genre_engagement).show(truncate=False)


+---------------------+----------+------------------+---------------------+------------------+------------------+---------------------+
|genre                |game_count|avg_playtime_hours|median_playtime_hours|p75_playtime_hours|p25_playtime_hours|stddev_playtime_hours|
+---------------------+----------+------------------+---------------------+------------------+------------------+---------------------+
|Massively Multiplayer|94        |51.34             |14.17                |49.83             |3.60              |114.98               |
|RPG                  |1217      |20.92             |7.60                 |21.28             |2.97              |59.21                |
|Strategy             |1083      |19.03             |7.40                 |20.83             |2.90              |36.12                |
|Early Access         |304       |20.63             |6.85                 |21.47             |2.68              |39.65                |
|Simulation           |1236      |32.52         


# Summary

## Query Description:
This query analyzes player engagement patterns across different game genres by:
1. Parsing the genres column (comma-separated list)
2. Calculating statistical metrics (mean, median, quartiles, standard deviation) for playtime per genre
3. Filtering genres with at least 50 games for statistical significance
4. Ordering results by median playtime (most engaging genres first)

## Dataset Characteristics:
- **Source:** ML-ready dataset (pre-cleaned)
- **Filtered:** Games with ≥500 reviews and price > 0 (paid games only)
- **Metrics:** Playtime in hours (converted from minutes)


In [16]:
spark.stop()